In [1]:
# =========================
# Install required libraries
# =========================

In [2]:
!pip install langgraph langchain-google-genai wikipedia requests


In [3]:
# =========================
# Imports
# =========================

In [4]:
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
import wikipedia
import requests
import os
from datetime import datetime
import re
import string


In [5]:
# =========================
# Step 1: Set API Keys
# =========================

In [6]:
os.environ["GOOGLE_API_KEY"] = "your api key"   # Replace with your Gemini API key
OPENWEATHER_API_KEY = "your api key"            # Replace with your OpenWeather API key


In [7]:
# =========================
# Step 2: Define initial chatbot state
# =========================



In [8]:
def get_initial_state():
    """Initial chatbot state with memory"""
    return {
        "messages": [],   # conversation history
        "city": None      # store last city for weather queries
    }

In [9]:
# =========================
# Step 3: Define Tools
# =========================


In [10]:
@tool
def calculator(expression: str) -> str:
    """Evaluate a math expression"""
    try:
        return str(eval(expression))
    except Exception:
        return "Sorry, I couldn't calculate that."

@tool
def wiki_search(query: str) -> str:
    """Search Wikipedia and return summary"""
    try:
        return wikipedia.summary(query, sentences=2)
    except Exception:
        return "No results found on Wikipedia."

@tool
def weather(city: str) -> str:
    """Get current weather for a city"""
    try:
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPENWEATHER_API_KEY}&units=metric"
        data = requests.get(url).json()
        if data.get("cod") != 200:
            return "City not found."
        temp = data["main"]["temp"]
        desc = data["weather"][0]["description"]
        humidity = data["main"]["humidity"]
        wind = data["wind"]["speed"]
        return (f"Weather in {city}:\n"
                f"Condition: {desc}\n"
                f"Temperature: {temp}°C\n"
                f"Humidity: {humidity}%\n"
                f"Wind: {wind} m/s")
    except Exception:
        return "Couldn't fetch weather right now."

@tool
def get_datetime(_: str = "") -> str:
    """Get current date and time"""
    now = datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")


/opt/anaconda3/lib/python3.13/site-packages/pydantic/main.py:1631: RuntimeWarning: fields may not start with an underscore, ignoring "_"
  warnings.warn(f'fields may not start with an underscore, ignoring "{f_name}"', RuntimeWarning)


In [11]:
# =========================
# Step 4: Define chatbot node with memory + tools + clarification
# =========================


In [12]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

def chatbot_node(state):
    user_input = state["input"].strip()
    messages = state["messages"]
    messages.append(("user", user_input))

    # ---- Calculator ----
    if user_input.lower().startswith("calc "):
        result = calculator.invoke(user_input[5:])
        response_text = f"Calculator result: {result}"

    # ---- Wikipedia ----
    elif user_input.lower().startswith("wiki "):
        result = wiki_search.invoke(user_input[5:])
        response_text = f"Wikipedia: {result}"

    # ---- Weather ----
    elif "weather" in user_input.lower():
        # Clean input
        clean_input = user_input.translate(str.maketrans('', '', string.punctuation))

        # Try to extract city from input
        city_match = re.search(r"weather in ([a-zA-Z\s]+)", clean_input, re.IGNORECASE)
        if city_match:
            city_name = city_match.group(1).strip()
            state["city"] = city_name  # store city in memory
        elif state.get("city"):
            city_name = state["city"]  # use previously stored city
        else:
            # Ask user for city if missing
            response_text = "I need your location to check the weather. Which city are you interested in?"
            messages.append(("assistant", response_text))
            return {"messages": messages, "output": response_text}

        # Fetch weather
        response_text = weather.invoke(city_name)

         # ---- Date & Time ----
    elif "time" in user_input.lower() or "date" in user_input.lower():
        result = get_datetime.invoke("")
        response_text = f"Current date & time: {result}"

    # ---- Default LLM ----
    else:
        response = llm.invoke(messages)
        response_text = response.content

    # Save assistant response
    messages.append(("assistant", response_text))
    return {"messages": messages, "output": response_text}



In [13]:
# =========================
# Step 5: Build LangGraph workflow
# =========================



In [14]:
graph = StateGraph(dict)
graph.add_node("chatbot", chatbot_node)
graph.set_entry_point("chatbot")
graph.add_edge("chatbot", END)

app = graph.compile()



In [15]:
# =========================
# Step 6: Chat loop
# =========================


In [ ]:
print("Smart Chatbot is ready! (type 'exit' to quit)\n")
state = get_initial_state()

while True:
    user_input = input("You: ").strip()
    if user_input.lower() in ["exit", "quit"]:
        print("Chatbot: Goodbye! 👋")
        break

    state["input"] = user_input
    response = app.invoke(state)
    print("Chatbot:", response["output"])
    

Smart Chatbot is ready! (type 'exit' to quit)



You:  hi


Chatbot: Hi there! How can I help you today?


You:  what's weather in palakkad


Chatbot: Weather in palakkad:
Condition: scattered clouds
Temperature: 24.7°C
Humidity: 93%
Wind: 3.54 m/s


...

Chatbot: That's a fantastic question, but the "best car" really depends on what you're looking for! There's no single "best" car for everyone because needs and preferences vary so much.

To help me give you a more tailored recommendation, could you tell me a bit about what's important to you? For example:

*   **What's your budget?** (e.g., under $20k, $30-50k, luxury range)
*   **What will you use it for primarily?** (e.g., daily commute, family trips, off-roading, performance driving, city driving)
*   **How many people will typically be in the car?** (e.g., just yourself, a couple, a family of four, larger group)
*   **What's important to you in a car?** (e.g., fuel efficiency, safety, reliability, cargo space, luxury, technology, performance, environmental impact)
*   **What kind of car do you prefer?** (e.g., sedan, SUV, hatchback, truck, electric vehicle)

Once I have a better idea of your priorities, I can suggest some categories or specific models that might be a good fit!


You:  best cafe in my area


Chatbot: I'd love to help you find the best cafe, but I need to know **your area** first!

Could you please tell me:

*   **Your city and state/region?**
*   **Or even a specific neighborhood or postal code?**

Once I have that information, I can try to give you some recommendations!


You:  what is the sum


Chatbot: The sum is the result of adding two or more numbers together.

For example:
*   The sum of 2 and 3 is 5 (2 + 3 = 5).
*   The sum of 10, 20, and 5 is 35 (10 + 20 + 5 = 35).

Do you have some numbers you'd like me to sum for you?
